In [1]:
from joblib import load
import os

# Load the models
model_gender = load(os.path.join('trials\\log_mel_energy_Gender_Age\\Gender\\model.joblib'))
model_age_male = load(os.path.join('trials\\log_mel_energy_Gender_Age\\Age_Male\\model.joblib'))
model_age_female = load(os.path.join('trials\\log_mel_energy_Gender_Age\\Age_Female\\model.joblib'))


print("Models loaded successfully.")

Models loaded successfully.


d:\ProgramFiles\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.2.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [2]:
X_train = load(os.path.join('trials\\features\\48k_mfcc_extra_hfcc_extra\\X_train.joblib'))
X_train2 = load(os.path.join('trials\\features\\48k_log_mel_energy_all_data\\X_train.joblib'))
y_train = load(os.path.join('trials\\features\\48k_mfcc_extra_hfcc_extra\\y_train.joblib'))

X_test = load(os.path.join('trials\\features\\48k_mfcc_extra_hfcc_extra\\X_test.joblib'))
X_test2 = load(os.path.join('trials\\features\\48k_log_mel_energy_all_data\\X_test.joblib'))
y_test = load(os.path.join('trials\\features\\48k_mfcc_extra_hfcc_extra\\y_test.joblib'))

In [3]:
import numpy as np
X_train_combined = np.concatenate((X_train[:,:150], X_train[:,7*75:9*75], X_train2[:,-75:]), axis=1)
X_test_combined = np.concatenate((X_test[:,:150], X_test[:,7*75:9*75], X_test2[:,-75:]), axis=1)
print(X_train_combined.shape)
print(X_test_combined.shape)

(172786, 375)
(9094, 375)


In [6]:
from sklearn.base import BaseEstimator, ClassifierMixin
import numpy as np
import time
from tqdm import tqdm

class GenderAgeModel(BaseEstimator, ClassifierMixin):
    def __init__(self, gender_model, male_age_model, female_age_model):
        self.gender_model = gender_model
        self.male_age_model = male_age_model
        self.female_age_model = female_age_model
        

    def predict_one(self, X):
        # Collect predictions from all models
        gender_pred = self.gender_model.predict(X)
        if gender_pred == 0:
            age_pred = self.male_age_model.predict(X)
            return age_pred
        else:
            age_pred = self.female_age_model.predict(X)
            return age_pred

        
    def predict(self, X):
        # Apply predict_one for each sample in X
        return np.array([self.predict_one(sample.reshape(1, -1)) for sample in tqdm(X, desc="Predicting")])
# Create an instance of the combined model
combined_model = GenderAgeModel(gender_model=model_gender, male_age_model=model_age_male , female_age_model=model_age_female)
# Measure the time taken for one prediction
start_time = time.time()
combined_model.predict(X_val_fold[:1])  # Predict for one sample
end_time = time.time()

print(f"Time taken for one prediction: {end_time - start_time:.6f} seconds")


Predicting: 100%|██████████| 1/1 [00:00<00:00, 71.42it/s]

Time taken for one prediction: 0.015997 seconds


In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


skf = StratifiedKFold(n_splits=20, shuffle=True, random_state=42)
train_idx, val_idx = next(skf.split(X_train_combined, y_train))

X_train_fold, X_val_fold = X_train_combined[train_idx], X_train_combined[val_idx]
y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]
X_train_balanced, y_train_balanced = X_train_fold, y_train_fold


y_val_pred = combined_model.predict(X_val_fold)
#y_val_pred_proba_3 = clf.predict_proba(X_val_fold)


Predicting:   7%|▋         | 564/8640 [00:06<01:27, 91.86it/s] 


KeyboardInterrupt: 

In [ ]:
print('Accuracy: %.9f' % (accuracy_score(y_val_fold, y_val_pred)))
print(classification_report(y_val_fold, y_val_pred))
print(confusion_matrix(y_val_fold, y_val_pred))

Accuracy: 0.996
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5907
           1       1.00      1.00      1.00      1029
           2       0.97      1.00      0.98       922
           3       1.00      1.00      1.00       782

    accuracy                           1.00      8640
   macro avg       0.99      1.00      1.00      8640
weighted avg       1.00      1.00      1.00      8640

[[5881    0   26    0]
 [   0 1028    0    1]
 [   4    0  918    0]
 [   0    0    0  782]]


In [9]:
#Predict on the test set
y_test_pred = combined_model.predict(X_test_combined)

# Print accuracy
print('Test Accuracy: %.9f' % (accuracy_score(y_test, y_test_pred)))

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_test_pred))

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))

Predicting: 100%|██████████| 9094/9094 [01:46<00:00, 85.38it/s] 

Test Accuracy: 0.978997141
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99      6217
           1       0.97      0.97      0.97      1084
           2       0.98      0.92      0.95       971
           3       0.97      0.97      0.97       822

    accuracy                           0.98      9094
   macro avg       0.97      0.96      0.97      9094
weighted avg       0.98      0.98      0.98      9094

Confusion Matrix:
[[6163   22   19   13]
 [  16 1053    1   14]
 [  76    2  892    1]
 [  13   12    2  795]]
